In [1]:
from appgeopy import *
from my_packages import *

In [ ]:
mainfolder = r"D:\1000_SCRIPTS\002_Manuscript_1"
input_mfile = os.path.join(mainfolder, "MFILE_Goldstein_Filtered_Inteferograms_list.txt")
prefix = "Choushui"

gps_los_difference_fpath = r"D:\1000_SCRIPTS\002_Manuscript_1\Differencing_GPS_LOS_mm_Manuscript001_v1.xlsx"
los_difference_savename = "InSAR_GPS_VERT_Difference.xlsx"
los_merge_savename = "InSAR_GPS_VERT_Merge.xlsx"
radius_of_buffer = 1000

validation_folder = os.path.join(mainfolder, "17_VERTICAL")

# o_displacement_folder = os.path.join(validation_folder, f"LOS_output_{kriging_folder_number}")
o_displacement_fpath = os.path.join(validation_folder, f"CALIBRATED_o{prefix}_DISPLACEMENT_VERT_mm.pkl")

los_difference_savepath = os.path.join(validation_folder, "VALIDATE_" + los_difference_savename)
los_merge_savepath = os.path.join(validation_folder, "VALIDATE_" + los_merge_savename)

# Load and configure data
o_displacement_geodf = pd.read_pickle(o_displacement_fpath, compression="zip").set_index("PointKey")
mfile_datetimes = datetime_handle.extract_datetime_from_mfile(input_mfile)[1:]
mutual_datetimes = sorted(set(mfile_datetimes).intersection(set(o_displacement_geodf.columns)))

select_pointkey_fpath = glob(os.path.join(validation_folder, f"*{radius_of_buffer}*.pkl"))[0]
select_pointkey = pd.read_pickle(select_pointkey_fpath, compression="zip")
available_stations = select_pointkey["STATION"].unique()

In [9]:
# Process each GPS station and compare LOS displacements
for select_station in tqdm(available_stations[:], desc="Processing GPS stations"):
    try:
        pointkey_by_station = select_pointkey.query("STATION==@select_station")["PointKey"]
        psc_by_station = o_displacement_geodf.loc[pointkey_by_station, mutual_datetimes]
        avg_deform_psc_by_station = pd.DataFrame(psc_by_station.mean(axis=0), columns=["U(mm)"])
        avg_deform_psc_by_station.index = avg_deform_psc_by_station.index.map(datetime_handle.convert_to_datetime)

        gps_los_difference_by_station = pd.read_excel(
            gps_los_difference_fpath, sheet_name=select_station, usecols=[0, 3], parse_dates=[0], index_col=[0]
        )

        mutual_index = datetime_handle.intersect_time_index(
            df1_index=avg_deform_psc_by_station.index, df2_index=gps_los_difference_by_station.index
        )

        mutual_index = pd.Index(mutual_index)
        psc_by_idx = avg_deform_psc_by_station.loc[mutual_index]
        gps_by_idx = gps_los_difference_by_station.loc[mutual_index]
        
        # Calculate the difference in LOS displacements
        los_difference = gps_by_idx - psc_by_idx
        
        gps_by_idx = gps_by_idx.rename(columns={"U(mm)":"GPS(mm)"})
        psc_by_idx = psc_by_idx.rename(columns={"U(mm)":"InSAR(mm)"})

        los_merge = pd.concat([gps_by_idx, psc_by_idx], axis=1)

        for dataframe in [los_merge, los_difference]:
            dataframe.reset_index(drop=False, inplace=True)
            dataframe.rename(columns={"index": "Datetime"}, inplace=True)

        data_io.save_df_to_excel(
            df_to_save=los_difference,
            filepath=los_difference_savepath,
            sheet_name=select_station,
            verbose=False,
        )

        data_io.save_df_to_excel(
            df_to_save=los_merge,
            filepath=los_merge_savepath,
            sheet_name=select_station,
            verbose=False,
        )

    except Exception as e:
        print(f"Error processing station {select_station}: {e}")
        continue

Processing GPS stations:   0%|          | 0/1 [00:00<?, ?it/s]